# Lectura de ficheros easyMoney

## Import Libraries

In [44]:
import numpy as np 
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

In [45]:
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format

In [47]:
from sklearn import preprocessing

## Import Files

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Products

In [49]:
ruta = '/content/drive/MyDrive/Capstone'

In [50]:
ca_file = ruta + '/' + 'commercial_activity_df.csv'

In [51]:
products_file =  ruta + '/' + 'products_df.csv'

In [52]:
sd_file =  ruta + '/' + 'sociodemographic_final.csv'

In [53]:
commercial = pd.read_csv(ca_file)

In [54]:
products = pd.read_csv(products_file)

In [55]:
sociodemographic = pd.read_csv(sd_file,sep=',')


para entry_date se hace el replace para los años no biciestos que tienen en febrero 29 días, se cambia a 28, ya que el dataset solo son días finales


In [56]:
commercial["entry_date"].replace('2015-02-29','2015-02-28', inplace=True)
commercial["entry_date"].replace('2019-02-29','2019-02-28', inplace=True)

pk_partition to date

In [57]:
sociodemographic ["pk_partition"] = pd.to_datetime(sociodemographic["pk_partition"], format = "%Y-%m-%d")
products ["pk_partition"] = pd.to_datetime(products["pk_partition"], format = "%Y-%m-%d")


commercial ["pk_partition"] = pd.to_datetime(commercial["pk_partition"], format = "%Y-%m-%d")
commercial ["entry_date"] = pd.to_datetime(commercial["entry_date"], format = "%Y-%m-%d")


In [58]:
commercial.head(5)

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01-28,2018-01-12,KHL,1.00,02 - PARTICULARES
1,1,1050611,2018-01-28,2015-08-10,KHE,0.00,03 - UNIVERSITARIO
2,2,1050612,2018-01-28,2015-08-10,KHE,0.00,03 - UNIVERSITARIO
3,3,1050613,2018-01-28,2015-08-10,KHD,0.00,03 - UNIVERSITARIO
4,4,1050614,2018-01-28,2015-08-10,KHE,1.00,03 - UNIVERSITARIO


In [59]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Unnamed: 0          int64         
 1   pk_cid              int64         
 2   pk_partition        datetime64[ns]
 3   short_term_deposit  int64         
 4   loans               int64         
 5   mortgage            int64         
 6   funds               int64         
 7   securities          int64         
 8   long_term_deposit   int64         
 9   em_account_pp       int64         
 10  credit_card         int64         
 11  payroll             float64       
 12  pension_plan        float64       
 13  payroll_account     int64         
 14  emc_account         int64         
 15  debit_card          int64         
 16  em_account_p        int64         
 17  em_acount           int64         
dtypes: datetime64[ns](1), float64(2), int64(15)
memory usage: 818.9 MB


In [60]:
#redefinimos el index con la columna Unnamed

commercial.set_index('Unnamed: 0', inplace = True)

In [61]:
#redefinimos el index con la columna Unnamed

products.set_index('Unnamed: 0', inplace = True)

In [62]:
#redefinimos el index con la columna Unnamed

sociodemographic.set_index('Unnamed: 0', inplace = True)

In [63]:
#products.head(3)

In [64]:
# clientes unicos
products['pk_cid'].nunique()

456373

In [65]:
#Vemos que las fechas van desde el 1 de enero del 2018 hasta el 28 de mayo del 2019. Es decir 18 meses y no hay valores nulos
#podemos analizar la permanencia de los clientes
#los clientes nuevos por mes y año
#los clientes que se fueron 
#el producto mas adquirido por mes y año
products['pk_partition'].value_counts(dropna=False)

2019-05-28    442995
2019-04-28    439627
2019-03-28    436183
2019-02-28    431727
2019-01-28    426875
2018-12-28    422481
2018-11-28    416387
2018-10-28    402300
2018-09-28    375323
2018-08-28    352922
2018-07-28    339339
2018-06-28    252104
2018-05-28    249926
2018-04-28    247463
2018-03-28    245258
2018-02-28    242521
2018-01-28    239493
Name: pk_partition, dtype: int64

In [66]:
#Chequeamos y rellenamos valores nulos

print(products.isnull().sum().to_string())

pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0


In [67]:
products['payroll'].fillna(0, inplace=True)

In [68]:
 products['pension_plan'].fillna(0, inplace=True)

In [69]:
print(products.isnull().sum().to_string())

pk_cid                0
pk_partition          0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
em_account_pp         0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_account_p          0
em_acount             0


In [70]:
#products[products['pk_cid'] == 1375586]

In [71]:
#products[products['pk_partition'] == '2019-05-28'].describe().T

### Generamos valores de diferencias

In [72]:
products_sorted = products.sort_values(by=['pk_cid', 'pk_partition'])

In [73]:
#products_sorted.head(20)

In [74]:
#Que estado tenia en el mes anterior
products_sorted['debit_card_previous_month'] = products_sorted.groupby('pk_cid')['debit_card'].shift(1)

In [75]:
products_sorted['debit_card_previous_month'].isnull().sum()

456373

In [76]:
lista_productos = ['short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount'] 

In [77]:
products.shape

(5962924, 17)

In [78]:
for productos in lista_productos:
 products_sorted[productos + '_previous_month' ] = products_sorted.groupby('pk_cid')[productos].shift(1) 
 products_sorted[productos + '_previous_month'].fillna(0, inplace=True)
 products_sorted['dif_' + productos] = products_sorted[productos] - products_sorted[productos + '_previous_month']

In [79]:
#products_sorted.head()

In [80]:
#Venta o cancelacion
products_sorted['dif_debit_card'] = products_sorted['debit_card'] - products_sorted['debit_card_previous_month']

In [81]:
products_sorted.shape

(5962924, 45)

In [82]:
products_sorted.columns

Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount',
       'debit_card_previous_month', 'short_term_deposit_previous_month',
       'dif_short_term_deposit', 'loans_previous_month', 'dif_loans',
       'mortgage_previous_month', 'dif_mortgage', 'funds_previous_month',
       'dif_funds', 'securities_previous_month', 'dif_securities',
       'long_term_deposit_previous_month', 'dif_long_term_deposit',
       'em_account_pp_previous_month', 'dif_em_account_pp',
       'credit_card_previous_month', 'dif_credit_card',
       'pension_plan_previous_month', 'dif_pension_plan',
       'payroll_account_previous_month', 'dif_payroll_account',
       'emc_account_previous_month', 'dif_emc_account', 'dif_debit_card',
       'em_account_p_previous_month', 'dif_em_account_p',
 

In [83]:
products_sorted['dif_debit_card'].value_counts()

0.00     5808360
1.00       99040
-1.00      55524
Name: dif_debit_card, dtype: int64

In [84]:
#Por mes cuantos clientes dejaron de tener el producto (-1)
def clientes_baja_productos(df, column):
 return df[df['dif_'+ column] == -1].pivot_table(index='pk_partition', values= 'dif_' + column, aggfunc=[len])
  

In [85]:
#Aca probamos con tarjeta de debito, despues continuamos con los otros productos y con aquellos que se dieron de alta. 
#clientes_baja_productos(products_sorted, 'debit_card')

In [86]:
 products_sorted[products_sorted['dif_debit_card'] == -1].pivot_table(index='pk_partition', values='dif_debit_card', aggfunc=[len])

,len
,dif_debit_card
pk_partition,
2018-02-28,2913
2018-03-28,2432
2018-04-28,2742
2018-05-28,3013
2018-06-28,2754
2018-07-28,3373
2018-08-28,4010
2018-09-28,2792


In [87]:
for producto in lista_productos:
  products[producto] = products[producto].astype('category')

In [88]:
products.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 13647308
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  category      
 3   loans               category      
 4   mortgage            category      
 5   funds               category      
 6   securities          category      
 7   long_term_deposit   category      
 8   em_account_pp       category      
 9   credit_card         category      
 10  payroll             float64       
 11  pension_plan        category      
 12  payroll_account     category      
 13  emc_account         category      
 14  debit_card          category      
 15  em_account_p        category      
 16  em_acount           category      
dtypes: category(14), datetime64[ns](1), float64(1), int64(1)
memory usage: 261.6 MB


### se agrega una columna de renevue por cada producto, y la columna de total revenue

- 10€ por cada cuenta vendida
- 40€ por los productos de ahorro e inversión (planes, fondos, etc.) 
- 60€ por los de financiación (prestamos y tarjetas).
- Tarjeta de credito es un préstamo. Me da 60 euros
- Tarjeta debito es un servicio (considerar 10 euros)


In [89]:
lista_servicios = ['em_account_pp',
        'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount']

lista_ahorro_inv = ['short_term_deposit',
       'funds', 'securities', 'long_term_deposit', 'pension_plan',]

lista_fincanciacion = [ 'loans', 'mortgage','credit_card', ]

In [90]:
products_sorted['total_revenue'] = 0
for productos in lista_productos:
  if productos in lista_servicios:
    revenue = 10
  elif productos in lista_ahorro_inv:
    revenue = 40
  elif productos in lista_fincanciacion:
    revenue = 60
  products_sorted['revenue_' + productos] = 0
  products_sorted.loc[products_sorted[productos] == 1 ,'revenue_' + productos]=  revenue
  products_sorted['total_revenue'][products_sorted[productos] == 1] = products_sorted['total_revenue'] [products_sorted[productos] == 1] + revenue
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [91]:
products_sorted['total_productos'] = 0
products_sorted['total_servicios'] = 0
products_sorted['total_ahorro_inv'] = 0
products_sorted['total_fincanciacion'] = 0

for productos in lista_productos:
  if productos in lista_servicios:
    products_sorted['total_servicios'] = products_sorted['total_servicios'] + products_sorted[productos]
  elif productos in lista_ahorro_inv:
    products_sorted['total_ahorro_inv'] = products_sorted['total_ahorro_inv'] + products_sorted[productos]
  elif productos in lista_fincanciacion:
    products_sorted['total_fincanciacion'] = products_sorted['total_fincanciacion'] + products_sorted[productos]

products_sorted['total_productos']  = products_sorted['total_servicios'] + products_sorted['total_ahorro_inv'] + products_sorted['total_fincanciacion']

In [92]:
#products_sorted[products_sorted['pk_cid']==16063]

In [93]:
#products_sorted

In [94]:
#products_sorted[products_sorted['pk_cid'] == 17457]

In [95]:
df_cliente = products_sorted[products_sorted['pk_cid'] == 17457]

In [96]:
#df_cliente.to_csv(r'/content/drive/MyDrive/Captstone/data/df_cliente_3.csv',index = False, sep=';')

In [97]:
# products_sorted.to_csv(r'/content/drive/MyDrive/Nuclio/Capstone/products_sorted.csv',index = False, sep=';')

In [98]:
df = products_sorted[products_sorted['total_revenue']>0]

In [99]:
df_filtrado = pd.DataFrame(products_sorted[products_sorted['total_revenue']>0])

In [100]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4841417 entries, 4078994 to 13336847
Data columns (total 64 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   pk_cid                             int64         
 1   pk_partition                       datetime64[ns]
 2   short_term_deposit                 int64         
 3   loans                              int64         
 4   mortgage                           int64         
 5   funds                              int64         
 6   securities                         int64         
 7   long_term_deposit                  int64         
 8   em_account_pp                      int64         
 9   credit_card                        int64         
 10  payroll                            float64       
 11  pension_plan                       float64       
 12  payroll_account                    int64         
 13  emc_account                        int64         


In [101]:
df_filtrado.shape

(4841417, 64)

In [102]:
lista_categorical = ['short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount',
       'debit_card_previous_month', 'short_term_deposit_previous_month',
       'dif_short_term_deposit', 'loans_previous_month', 'dif_loans',
       'mortgage_previous_month', 'dif_mortgage', 'funds_previous_month',
       'dif_funds', 'securities_previous_month', 'dif_securities',
       'long_term_deposit_previous_month', 'dif_long_term_deposit',
       'em_account_pp_previous_month', 'dif_em_account_pp',
       'credit_card_previous_month', 'dif_credit_card',
       'pension_plan_previous_month', 'dif_pension_plan',
       'payroll_account_previous_month', 'dif_payroll_account',
       'emc_account_previous_month', 'dif_emc_account', 'dif_debit_card',
       'em_account_p_previous_month', 'dif_em_account_p',
       'em_acount_previous_month', 'dif_em_acount','total_productos','total_servicios','total_ahorro_inv','total_fincanciacion']

       #, 
       #'revenue_short_term_deposit', 'revenue_loans', 'revenue_mortgage',
       #'revenue_funds', 'revenue_securities', 'revenue_long_term_deposit',
       #'revenue_em_account_pp', 'revenue_credit_card', 'revenue_pension_plan',
       #'revenue_payroll_account', 'revenue_emc_account', 'revenue_debit_card',
       #'revenue_em_account_p', 'revenue_em_acount' 

In [103]:
for producto in lista_categorical:
  df_filtrado[producto] = df_filtrado[producto].astype('category')

In [104]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4841417 entries, 4078994 to 13336847
Data columns (total 64 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   pk_cid                             int64         
 1   pk_partition                       datetime64[ns]
 2   short_term_deposit                 category      
 3   loans                              category      
 4   mortgage                           category      
 5   funds                              category      
 6   securities                         category      
 7   long_term_deposit                  category      
 8   em_account_pp                      category      
 9   credit_card                        category      
 10  payroll                            category      
 11  pension_plan                       category      
 12  payroll_account                    category      
 13  emc_account                        category      


In [105]:
commercial['active_customer'] = commercial['active_customer'].astype('category')

In [106]:
products = []
df = []

In [107]:
new_df_filtrado = pd.merge(df_filtrado,commercial,how='left',on=["pk_cid", "pk_partition"])

In [108]:
new_df_filtrado.shape

(4841417, 68)

In [109]:
commercial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 13647308
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   pk_cid           int64         
 1   pk_partition     datetime64[ns]
 2   entry_date       datetime64[ns]
 3   entry_channel    object        
 4   active_customer  category      
 5   segment          object        
dtypes: category(1), datetime64[ns](2), int64(1), object(2)
memory usage: 278.6+ MB


In [110]:
new_df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4841417 entries, 0 to 4841416
Data columns (total 68 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   pk_cid                             int64         
 1   pk_partition                       datetime64[ns]
 2   short_term_deposit                 category      
 3   loans                              category      
 4   mortgage                           category      
 5   funds                              category      
 6   securities                         category      
 7   long_term_deposit                  category      
 8   em_account_pp                      category      
 9   credit_card                        category      
 10  payroll                            category      
 11  pension_plan                       category      
 12  payroll_account                    category      
 13  emc_account                        category      
 14  de

In [111]:
sociodemographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 13647308
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   country_id          object        
 3   region_code         float64       
 4   age                 int64         
 5   salary              float64       
 6   Population_region   int64         
 7   Region_name         object        
 8   Population_country  int64         
 9   Population_Roc      float64       
 10  deseased            int64         
 11  gender_H            int64         
dtypes: datetime64[ns](1), float64(3), int64(6), object(2)
memory usage: 591.4+ MB


In [112]:
sociodemographic['deseased'] = sociodemographic['deseased'].astype('category')
sociodemographic['gender_H'] = sociodemographic['gender_H'].astype('category')

sociodemographic['country_id'] = sociodemographic['country_id'].astype('category')

sociodemographic['age']= sociodemographic['age'].astype('uint8')

# Cual es el valor para los nulos?
#sociodemographic['region_code'] = sociodemographic['region_code'].astype('int64')
#sociodemographic['Population_Roc'] = sociodemographic['Population_Roc'].astype('int64')

In [113]:
sociodemographic.head()

,pk_cid,pk_partition,country_id,region_code,age,salary,Population_region,Region_name,Population_country,Population_Roc,deseased,gender_H
Unnamed: 0,,,,,,,,,,,,
0,1375586,2018-01-28,ES,29.00,35,"87,218.10",1695651,Málaga,46754778,NaN,0,1
1,1050611,2018-01-28,ES,13.00,23,"35,548.74",492591,Ciudad Real,46754778,NaN,0,0
2,1050612,2018-01-28,ES,13.00,23,"122,179.11",492591,Ciudad Real,46754778,NaN,0,0
3,1050613,2018-01-28,ES,50.00,22,"119,775.54",967452,Zaragoza,46754778,NaN,0,1
4,1050614,2018-01-28,ES,50.00,23,"88,654.65",967452,Zaragoza,46754778,NaN,0,0


In [114]:
final_df = pd.merge(new_df_filtrado,sociodemographic,how='left',on=["pk_cid", "pk_partition"])

In [115]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4841417 entries, 0 to 4841416
Data columns (total 78 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   pk_cid                             int64         
 1   pk_partition                       datetime64[ns]
 2   short_term_deposit                 category      
 3   loans                              category      
 4   mortgage                           category      
 5   funds                              category      
 6   securities                         category      
 7   long_term_deposit                  category      
 8   em_account_pp                      category      
 9   credit_card                        category      
 10  payroll                            category      
 11  pension_plan                       category      
 12  payroll_account                    category      
 13  emc_account                        category      
 14  de

In [116]:
final_df.shape

(4841417, 78)

In [161]:
final_df['total_fincanciacion']

TypeError: ignored

In [117]:
#final_df.to_csv(r'/content/drive/MyDrive/Captstone/data/final_df.csv',index = False, sep=';')

##Entry Channel View


In [118]:
df_cliente_date = final_df[['pk_cid','entry_date']].groupby(['pk_cid','entry_date']).count()
df_cliente_date.reset_index(inplace=True)
df_cliente_date_2 = df_cliente_date.groupby('pk_cid').count()

In [119]:
df_cliente_date_2.reset_index(inplace=True)

In [120]:
df_cliente_date_2[df_cliente_date_2['entry_date']>1]

,pk_cid,entry_date
342286,1534231,2
346406,1542479,2


In [121]:
final_df[final_df['pk_cid'].isin([1534231,1542479])][['pk_cid','entry_date','entry_channel']]

,pk_cid,entry_date,entry_channel
4825098,1534231,2019-02-23,NaN
4825099,1534231,2019-02-23,KHM
4825100,1534231,2019-02-22,KHM
4825101,1534231,2019-02-22,KHM
4835861,1542479,2019-03-31,KHM
4835862,1542479,2015-11-29,KHM


In [122]:
df_cliente_date_2 = df_cliente_date.groupby('pk_cid').count()

In [123]:
df_cliente_date = final_df[['pk_cid','entry_date','entry_channel']].groupby(['pk_cid','entry_date','entry_channel']).count()
df_cliente_date.reset_index(inplace=True)
df_cliente_date_2 = df_cliente_date.groupby('pk_cid').count()

In [124]:
df_cliente_date.groupby('pk_cid').count()

,entry_date,entry_channel
pk_cid,,
15891,1,1
16203,1,1
16502,1,1
17457,1,1
17799,1,1
...,...,...
1548208,1,1
1548210,1,1
1548211,1,1


In [ ]:
dftotal_fincanciacion                

In [125]:
df_cliente_date_2.reset_index(inplace=True)
df_cliente_date_2[df_cliente_date_2['entry_date']>1]


,pk_cid,entry_date,entry_channel
74,41264,2,2
109,50990,2,2
112,51819,2,2
126,56701,2,2
142,62418,2,2
...,...,...,...
344666,1541766,2,2
344739,1541919,2,2
344962,1542423,2,2
344992,1542479,2,2


In [126]:
del df_cliente_date_2
del df_cliente_date

#Fill Entry_Channel

In [127]:
df_channel = final_df[final_df['pk_cid'].isin(final_df[final_df['entry_channel'].isna()]['pk_cid'])][['pk_cid','pk_partition','entry_date','entry_channel']]

In [128]:
df_channel_2 = df_channel[~df_channel['entry_channel'].isna()][['pk_cid','pk_partition','entry_channel']].groupby('pk_cid').min()
df_channel_2.reset_index(inplace=True)
df_channel_2['entry_channel_na'] = df_channel_2['entry_channel']


In [129]:
df_channel_2.drop(['entry_channel'], axis=1, inplace=True)

In [130]:
df_channel_2 = pd.merge(final_df,df_channel_2,on=['pk_cid'], how='left')

In [131]:
# se llena la variable entry_channel
final_df.loc[final_df['entry_channel'].isna(),'entry_channel'] = df_channel_2[df_channel_2['entry_channel'].isna()]['entry_channel_na']
del df_channel_2
del df_channel

In [132]:
#df_channel_2[df_channel_2['pk_cid'].isin(df_channel_2[df_channel_2['entry_channel'].isna() & df_channel_2['entry_channel_na'].isna() ]['pk_cid'])]

In [133]:
final_df['entry_channel'].isna().sum()

3031

In [134]:
#llenamos entry_channel con la moda los que solo tienen una fecha
final_df['entry_channel'].fillna(final_df['entry_channel'].mode()[0], inplace=True)

In [135]:
final_df.groupby('entry_channel').count()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,country_id,region_code,age,salary,Population_region,Region_name,Population_country,Population_Roc,deseased,gender_H
entry_channel,,,,,,,,,,,,,,,,,,,,,
004,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,0,17,17
007,577,577,577,577,577,577,577,577,577,577,...,577,577,577,577,577,577,577,0,577,577
013,353,353,353,353,353,353,353,353,353,353,...,353,353,353,353,353,353,353,0,353,353
KAA,94,94,94,94,94,94,94,94,94,94,...,94,94,94,94,94,94,94,0,94,94
KAB,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,0,21,21
KAD,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,0,34,34
KAE,45,45,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,0,45,45
KAF,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,0,49,49
KAG,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,0,217,217


In [136]:
final_df.describe().T

,count,mean,std,min,25%,50%,75%,max
pk_cid,"4,841,417.00","1,225,907.75","157,327.08","15,891.00","1,105,456.00","1,218,073.00","1,337,464.00","1,553,571.00"
total_revenue,"4,841,417.00",15.56,16.70,10.00,10.00,10.00,10.00,260.00
revenue_short_term_deposit,"4,841,417.00",0.13,2.25,0.00,0.00,0.00,0.00,40.00
revenue_loans,"4,841,417.00",0.01,0.59,0.00,0.00,0.00,0.00,60.00
revenue_mortgage,"4,841,417.00",0.00,0.49,0.00,0.00,0.00,0.00,60.00
revenue_funds,"4,841,417.00",0.17,2.57,0.00,0.00,0.00,0.00,40.00
revenue_securities,"4,841,417.00",0.18,2.70,0.00,0.00,0.00,0.00,40.00
revenue_long_term_deposit,"4,841,417.00",0.83,5.71,0.00,0.00,0.00,0.00,40.00
revenue_em_account_pp,"4,841,417.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
revenue_credit_card,"4,841,417.00",0.88,7.21,0.00,0.00,0.00,0.00,60.00


In [ ]:
final_df[final_df['pk_cid']==1186021][['salary', 'age']]

In [137]:
final_df[final_df['pk_cid']==1186021][['salary', 'age']]

,salary,age
2041835,"88,664.97",21
2041836,"88,664.97",21
2041837,"88,664.97",21
2041838,"88,664.97",21
2041839,"88,664.97",21
2041840,"88,664.97",21
2041841,"88,664.97",21
2041842,"88,664.97",21
2041843,"88,664.97",22
2041844,"88,664.97",22


In [138]:
final_df[final_df['salary']	<= 88665]['pk_cid'].nunique()

233524

1.   agregar sociodemografico final, que tenga la columna region_name
2.   lo mismo para entry_channel, hacer con segment
3.   guardar como csv y avisar a Flor




In [139]:
final_df[['pk_cid','segment']].groupby(['pk_cid','segment'],dropna=False).count()


Empty DataFrame
Columns: []
Index: [(15891, nan), (16203, 01 - TOP), (16203, 02 - PARTICULARES), (16502, 02 - PARTICULARES), (17457, 02 - PARTICULARES), (17799, 02 - PARTICULARES), (17970, 02 - PARTICULARES), (17980, 01 - TOP), (18146, 01 - TOP), (18704, 02 - PARTICULARES), (19021, 02 - PARTICULARES), (19268, 01 - TOP), (19268, 02 - PARTICULARES), (19684, 02 - PARTICULARES), (20243, 02 - PARTICULARES), (20315, 02 - PARTICULARES), (20333, 02 - PARTICULARES), (20417, 02 - PARTICULARES), (20919, 02 - PARTICULARES), (21551, 02 - PARTICULARES), (21719, 02 - PARTICULARES), (22212, 02 - PARTICULARES), (22635, 02 - PARTICULARES), (22635, nan), (22648, 02 - PARTICULARES), (23434, 01 - TOP), (24415, 02 - PARTICULARES), (24415, nan), (24468, 02 - PARTICULARES), (24763, 02 - PARTICULARES), (25036, 02 - PARTICULARES), (25054, 02 - PARTICULARES), (25398, 02 - PARTICULARES), (25612, 01 - TOP), (25612, 02 - PARTICULARES), (26018, 01 - TOP), (26163, 02 - PARTICULARES), (28191, 02 - PARTICULARES), (28470, 02 - PARTICULARES), (29008, 02 - PARTICULARES), (29484, 02 - PARTICULARES), (29896, 02 - PARTICULARES), (29947, 01 - TOP), (31365, 02 - PARTICULARES), (31365, nan), (31381, 02 - PARTICULARES), (31624, 02 - PARTICULARES), (31657, 01 - TOP), (31657, nan), (31681, 01 - TOP), (31825, 01 - TOP), (31825, 02 - PARTICULARES), (31972, 02 - PARTICULARES), (32163, 01 - TOP), (32376, 01 - TOP), (32376, 02 - PARTICULARES), (32519, 02 - PARTICULARES), (32560, 02 - PARTICULARES), (32560, nan), (33046, 02 - PARTICULARES), (33205, 02 - PARTICULARES), (33205, nan), (33534, 01 - TOP), (33620, 02 - PARTICULARES), (33706, 02 - PARTICULARES), (33941, 02 - PARTICULARES), (35063, 01 - TOP), (35509, 01 - TOP), (35725, 02 - PARTICULARES), (35843, nan), (35849, 02 - PARTICULARES), (35992, 02 - PARTICULARES), (36050, 01 - TOP), (36050, 02 - PARTICULARES), (36050, nan), (36070, 01 - TOP), (36070, nan), (36148, 01 - TOP), (36148, 02 - PARTICULARES), (37182, 02 - PARTICULARES), (37182, nan), (37299, 01 - TOP), (37928, 01 - TOP), (37928, nan), (37948, 02 - PARTICULARES), (38211, 02 - PARTICULARES), (38433, 02 - PARTICULARES), (38433, nan), (39008, 02 - PARTICULARES), (39008, nan), (39997, 02 - PARTICULARES), (40486, 02 - PARTICULARES), (40698, 02 - PARTICULARES), (40896, 02 - PARTICULARES), (41264, 02 - PARTICULARES), (41264, nan), (43228, 02 - PARTICULARES), (43520, 02 - PARTICULARES), (43521, 02 - PARTICULARES), (43521, nan), ...]

[435312 rows x 0 columns]

###Variable Segment View



In [140]:
final_df['segment'].isna().sum()

76977

In [141]:
df_segment = final_df[final_df['pk_cid'].isin(final_df[final_df['segment'].isna()]['pk_cid'])].groupby('pk_cid',dropna=False).count()[['segment','pk_partition']]
df_segment.reset_index(inplace=True)

In [142]:
df_segment_1 = final_df[final_df['pk_cid'].isin(df_segment[df_segment['pk_partition']>1]['pk_cid'])].groupby('pk_cid',dropna=False).count()

In [143]:
df_segment_1['pk_partition'].min()

2

In [144]:
final_df.groupby('segment',dropna=False).count()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,country_id,region_code,age,salary,Population_region,Region_name,Population_country,Population_Roc,deseased,gender_H
segment,,,,,,,,,,,,,,,,,,,,,
01 - TOP,91463,91463,91463,91463,91463,91463,91463,91463,91463,91463,...,91463,91170,91463,91463,91463,91463,91463,293,91463,91463
02 - PARTICULARES,1361406,1361406,1361406,1361406,1361406,1361406,1361406,1361406,1361406,1361406,...,1361406,1360167,1361406,1361406,1361406,1361406,1361406,1239,1361406,1361406
03 - UNIVERSITARIO,3311571,3311571,3311571,3311571,3311571,3311571,3311571,3311571,3311571,3311571,...,3311571,3311299,3311571,3311571,3311571,3311571,3311571,272,3311571,3311571
NaN,76977,76977,76977,76977,76977,76977,76977,76977,76977,76977,...,76977,76972,76977,76977,76977,76977,76977,5,76977,76977


#Fill Segment

In [145]:
 df_channel = final_df[final_df['pk_cid'].isin(final_df[final_df['segment'].isna()]['pk_cid'])][['pk_cid','pk_partition','segment']]

In [146]:
df_channel_2 = df_channel[~df_channel['segment'].isna()][['pk_cid','pk_partition','segment']].groupby('pk_cid').min()
df_channel_2.reset_index(inplace=True)
df_channel_2['segment_na'] = df_channel_2['segment']

In [147]:
df_channel_2.drop(['segment'], axis=1, inplace=True)

In [148]:
df_channel_2 = pd.merge(final_df,df_channel_2,on=['pk_cid'], how='left')

In [149]:
# se llena la variable entry_channel
final_df.loc[final_df['segment'].isna(),'segment'] = df_channel_2[df_channel_2['segment'].isna()]['segment_na']
df_channel_2 = []

In [150]:
final_df['segment'].fillna(final_df['segment'].mode()[0], inplace=True)

In [151]:
del df_channel_2
del df_channel

#Salida CSV

In [152]:
final_df.to_csv(ruta + '/' + r'final_df.csv',index = False, sep=';')

In [153]:
final_df.to_pickle(ruta + '/' + r'final_df1', compression='infer', protocol=4, storage_options=None)

In [154]:
print(final_df.isnull().sum().to_string())

pk_cid                                     0
pk_partition                               0
short_term_deposit                         0
loans                                      0
mortgage                                   0
funds                                      0
securities                                 0
long_term_deposit                          0
em_account_pp                              0
credit_card                                0
payroll                                    0
pension_plan                               0
payroll_account                            0
emc_account                                0
debit_card                                 0
em_account_p                               0
em_acount                                  0
debit_card_previous_month                  0
short_term_deposit_previous_month          0
dif_short_term_deposit                     0
loans_previous_month                       0
dif_loans                                  0
mortgage_p